## Import the stuff we need


In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from collections import defaultdict
import numpy as np
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## Load the Google Sheet and store it in a dataframe


In [2]:
# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Get the path to your credentials file from the environment variable
creds_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Add your service account credentials file
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)

# Authorize the client
client = gspread.authorize(creds)

# Define the Google Sheet URL
sheet_url = os.getenv('GOOGLE_SHEET_URL')

# Extract the spreadsheet ID from the URL
spreadsheet_id = sheet_url.split('/d/')[1].split('/')[0]

# Open the Google Sheet by ID
spreadsheet = client.open_by_key(spreadsheet_id)
sheet = client.open_by_key(spreadsheet_id).sheet1  # or use .get_worksheet(index) for specific sheet

# Get all values in the sheet
data = sheet.get_all_records()

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data)

## Define the functions that will do the work


In [3]:
def split_into_balanced_groups(df, num_groups=None, employees_per_group=None, separate_remote=False):
    if num_groups is None and employees_per_group is None:
        raise ValueError("Either num_groups or employees_per_group must be specified.")
    
    # Filter out employees who are not attending
    df = df[df['Attending'] == 'Yes']

    # Determine the number of groups
    if num_groups is None:
        num_groups = int(np.ceil(len(df) / employees_per_group))
    
    # Calculate base group size and number of larger groups
    base_size = len(df) // num_groups
    larger_groups_count = len(df) % num_groups
    
    # Separate remote and non-remote if required
    if separate_remote:
        remote_df = df[df['Remote'] == 'Yes']
        non_remote_df = df[df['Remote'] == 'No']
        
        remote_groups = _create_balanced_groups(remote_df, num_groups, base_size, larger_groups_count)
        non_remote_groups = _create_balanced_groups(non_remote_df, num_groups, base_size, larger_groups_count)
        
        # Combine remote and non-remote groups
        groups = [r + nr for r, nr in zip(remote_groups, non_remote_groups)]
    else:
        groups = _create_balanced_groups(df, num_groups, base_size, larger_groups_count)
    
    return groups

def _create_balanced_groups(df, num_groups, base_size, larger_groups_count):
    # Shuffle the DataFrame to ensure randomness
    df = df.sample(frac=1).reset_index(drop=True)
    
    # Group by team
    team_groups = defaultdict(list)
    for _, row in df.iterrows():
        team_groups[row['Team']].append(row)
    
    # Create empty groups
    groups = [[] for _ in range(num_groups)]
    
    # Distribute team members evenly across groups
    for team, members in team_groups.items():
        for i, member in enumerate(members):
            target_group = i % num_groups
            if len(groups[target_group]) < base_size or (target_group < larger_groups_count and len(groups[target_group]) == base_size):
                groups[target_group].append(member)
            else:
                # Find the next available group
                for j in range(num_groups):
                    if len(groups[j]) < base_size or (j < larger_groups_count and len(groups[j]) == base_size):
                        groups[j].append(member)
                        break
    
    return groups

def groups_to_dataframe(groups):
    # Find the maximum group size
    max_group_size = max(len(group) for group in groups)
    
    # Create a dictionary to hold group data
    group_data = {f'Group {i+1}': [None] * max_group_size for i in range(len(groups))}
    
    # Fill the dictionary with group members
    for i, group in enumerate(groups):
        for j, member in enumerate(group):
            group_data[f'Group {i+1}'][j] = f"{member['Emails']} ({member['Team']}, {member['Remote']})"
    
    # Convert the dictionary to a DataFrame
    return pd.DataFrame(group_data)

def write_to_google_sheet(df, spreadsheet, sheet_name='Groups'):
    try:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(len(df)+1), cols=str(len(df.columns)))
    except gspread.exceptions.APIError:
        worksheet = spreadsheet.worksheet(sheet_name)
        worksheet.clear()
    
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

## Make the groups


In [4]:
groups = split_into_balanced_groups(df, num_groups=4, separate_remote=False)
group_df = groups_to_dataframe(groups)

## Write the groups to the Google Sheet


In [5]:
write_to_google_sheet(group_df, spreadsheet)